In [1]:
import pandas as pd
import numpy as np
import random
import collections
import string
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
df = pd.read_csv('Brand_db.csv')
df["Done"] = 0
df

,Id,Brand_name,Category,Sub_category,Done
0,0,Airtel (Bharti Air),Bills,Phone Bill,0
1,1,Jio,Bills,Phone Bill,0
2,2,Amul,Foods and Drinks,Snacks,0
3,3,Spencers,Groceries,Supermart,0
4,4,Reliance Fresh,Groceries,Supermart,0
5,5,Kwality Bazaar,Groceries,Small Grocery Shop,0
6,6,Shopper Stop,Shopping,Clothing Center,0
7,7,Reliance Digital,Shopping,Electronic Center,0
8,8,Louis Philippe,Shopping,M Clothing Brand,0
9,9,Arrow,Shopping,M Clothing Brand,0


In [4]:
list_merchants = ["UPI", "VPS", "ATM", "ACH", "MMT", "IIN", "IPS", "VIN", "CMS", "BIL", "NFS", "OTHER"]
dict_no_fields_narr = {"UPI": 5, "VPS": 5, "ATM": 4, "ACH": 4, "MMT": 6, "IIN": 5, "IPS": 5, "VIN": 4, "CMS": 3, "BIL": 5, "NFS": 4, "OTHER": 1}
already_list,already_list_alpha = [],[]
category_list = ["Bills", "Entertainment", "Foods and Drinks", "Fuel", "Groceries","Personal Care", "Shopping", "Travel"]
category_list_lower = ["Bills", "Foods and Drinks", "Fuel", "Groceries", "Personal Care", "Shopping",  "Travel"]
sub_category_list = df["Sub_category"].unique().tolist()
to_rem = ['Pharmacy', 'Payment App', 'Fuel', 'Hospital','Electricity', 'Game Shop', 'Toy Center', 'Wifi', 'Courier', 'Other', 'Hotel' ,'Spa Brand', 'Insurance', 'Cash Withdrawal' ,'Credit Card', 'Transfer', 'Investment', 'Loan (EMI)', 'Rent']
sub_list = sub_category_list.copy()
for i in to_rem:
    sub_list.remove(i)
sub_category_list = sorted(sub_category_list)

#Sub_list: contains only the sub categories on which it is suitable to apply the model 
sub_list = sorted(sub_list)

#counts = brand_df["Sub_category"].value_counts().tolist()
cnt = df.groupby(['Sub_category']).size().tolist()
sub_df = pd.DataFrame({"sub" : sorted(df["Sub_category"].unique().tolist()), "Count":cnt})
#print(brand_df["Sub_category"].value_counts())
#print(brand_df.groupby(['Sub_category']).size())
#print(brand_df["Sub_category"].unique().tolist())
#sub_category_df
#print(len(brand_df["Sub_category"].unique().tolist()))
#len(counts)

In [5]:
for s in to_rem:
    sub_df.drop(sub_df[sub_df['sub'] == s].index, inplace = True) 
    df.drop(df[df["Sub_category"]==s].index, inplace = True)

In [6]:
sub_df.reset_index(drop = True, inplace=True)
df.reset_index(drop = True, inplace = True)
sub_df

,sub,Count
0,Accessory Brand,4
1,Bag Brand,4
2,Bar,8
3,Cab Brand,3
4,Cinema,3
5,Cinema Booking Online,1
6,Clothing Center,4
7,Coffee Shop,3
8,Cosmetics,5
9,E-Pharmacy,2


In [7]:
sub_list = sub_df["sub"].tolist()
print(sub_list)

['Accessory Brand', 'Bag Brand', 'Bar', 'Cab Brand', 'Cinema', 'Cinema Booking Online', 'Clothing Center', 'Coffee Shop', 'Cosmetics', 'E-Pharmacy', 'Electronic Brand', 'Electronic Center', 'Electronic Store', 'Eyewear Brand', 'F Clothing Brand', 'F Footwear Brand', 'Fast Food', 'Food Order', 'Furniture', 'Game Center', 'Gym', 'Holiday Planner', 'Jewellery', 'Liquor', 'M Clothing Brand', 'M Footwear Brand', 'Phone Bill', 'Restaurant', 'Salon', 'Shopping Center', 'Shopping Center Online', 'Small Grocery Shop', 'Snacks', 'Sports Brand', 'Streaming Brand', 'Supermart', 'Supermart Online', 'TV', 'U Clothing Brand', 'U Footwear Brand', 'Vehicle Rental', 'Watch Brand']


In [10]:
total_brands = 0
for i in range(42):
    total_brands += sub_df.at[i,"Count"]
total_brands

215

In [11]:
counter_df = sub_df.copy()
counter_df["Count"]=0

In [12]:
df

,Id,Brand_name,Category,Sub_category,Done
0,0,Airtel (Bharti Air),Bills,Phone Bill,0
1,1,Jio,Bills,Phone Bill,0
2,2,Amul,Foods and Drinks,Snacks,0
3,3,Spencers,Groceries,Supermart,0
4,4,Reliance Fresh,Groceries,Supermart,0
5,5,Kwality Bazaar,Groceries,Small Grocery Shop,0
6,6,Shopper Stop,Shopping,Clothing Center,0
7,7,Reliance Digital,Shopping,Electronic Center,0
8,8,Louis Philippe,Shopping,M Clothing Brand,0
9,9,Arrow,Shopping,M Clothing Brand,0


In [13]:
# for s in sub_list:
#     print (s)
#     d = df[df["Sub_category"]==s]
#     d.to_json("sub/"+s+".json", orient = 'records')
    